In [27]:
import torch 
from torch import nn 
from typing import Dict, Any, Optional


# Epxert  -> FNN network -> thorugh token will get pass. This will create the Sparse Arch 
# Router  -> FFN network -> It will select the subset of form these experts and decide for epoch where should the token will go.



### Experts ### 

class ExpertLayer(nn.Module):
    
    '''This is basically when say number of layer ; we are 
    number of ExpertLayer or FNN so this will get if we 4 Expert then this get 
    repated 4 times'''
    
    def __init__(self,input_dim:int, output_dim: int, hidden_dim: Optional[int]=None):
        
        super().__init__()
        self.input_dim = input_dim # input neuron  
        self.output_dim = output_dim # output neuron 
        self.hidden_dim = hidden_dim or 4 * input_dim # input -> 20 -> 80
        
        self.network = nn.Sequential( # -> -> -> 
            
            nn.Linear(input_dim,self.hidden_dim), #self.hidden we are using this directly 
            nn.ReLU(),
            nn.Linear(self.hidden_dim,output_dim)
        )
       # print( self.network )
    def forward(self,x):
        return self.network(x)
    
    def get_config(self):
        return {   # just returining the dict with hyper-para 
            'input_dim' : self.input_dim,
            'hidden_dim': self.hidden_dim,
            'output_dim': self.output_dim
        }
        
        
class FFNExpert(ExpertLayer):
    
    def __init__(self, input_dim, output_dim, hidden_dim: Optional[int]=None, num_layer=4,
                 dropout = 0.2):
        
        super().__init__(input_dim,output_dim,hidden_dim)
        
        # res = []
        
        
        layers = [] #will have layers acc num_layer 
        current_dim = input_dim # 10 
        
        for _ in range(num_layer-1):  ## extending our layers 
            layers.extend([
                nn.Linear(input_dim,self.hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            current_dim = self.hidden_dim
            
        
            
        
        
            
        ## belwo number of layers going to single layer  
        layers.append(nn.Linear(current_dim, output_dim))
        #print(f'inisde the layer 1 :{layers}')
        
       # print(f'inisde the layer 2:{layers}')
        
        self.network = nn.Sequential(*layers) # what is happening here ?? 
        print(self.network) 
       # print(f'inside network after adding : {self.network}')
        
    
    def get_config(self):
        
        config = super().get_config()
        config.update({
            'num_layers': len(self.network) // 3+1 ,
            'dropout' :self.network[2].p if len(self.network) > 2 else 0.0
        })
        self.res.append(config)
        
    
        return config
    
    # def get_res(self):
    #     return print(self.res)
    
    
   
check = FFNExpert(input_dim= 10 , output_dim= 4, hidden_dim= 10)

x = torch.randn(1,10)
out = check(x)

print(out.shape)

    
    

Sequential(
  (0): Linear(in_features=10, out_features=10, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.2, inplace=False)
  (3): Linear(in_features=10, out_features=10, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.2, inplace=False)
  (6): Linear(in_features=10, out_features=10, bias=True)
  (7): ReLU()
  (8): Dropout(p=0.2, inplace=False)
  (9): Linear(in_features=10, out_features=4, bias=True)
)
torch.Size([1, 4])


In [25]:

print(check.get_res)

<bound method FFNExpert.get_res of FFNExpert(
  (network): Sequential(
    (0): Linear(in_features=10, out_features=10, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=10, out_features=10, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=10, out_features=10, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.2, inplace=False)
    (9): Linear(in_features=10, out_features=4, bias=True)
  )
)>
